# Notebook E-tivity 3 CE4021 Task 2

Student name: Phil Clarke

Student ID: 23291567

<hr style=\"border:2px solid gray\"> </hr>

## Imports

In [ ]:
#None

If you believe required imports are missing, please contact your moderator.

<hr style=\"border:2px solid gray\"> </hr>

## Task 2

Use the below information to create a Naive Bayes SPAM filter. Test your filter using the messages in new_emails. You may add as many cells as you require to complete the task.

In [18]:
previous_spam = ['send us your password', 'review our website', 'send your password', 'send us your account']
previous_ham = ['Your activity report','benefits physical activity', 'the importance vows']
new_emails = {'spam':['renew your password', 'renew your vows'], 'ham':['benefits of our account', 'the importance of physical activity']}

In [19]:
vocab_words_spam = []

for sentence in previous_spam:
    sentence_as_list = sentence.split()
    for word in sentence_as_list:
        vocab_words_spam.append(word)     

In [20]:
print(vocab_words_spam)
vocab_unique_words_spam = list(dict.fromkeys(vocab_words_spam))
print(vocab_unique_words_spam)

['send', 'us', 'your', 'password', 'review', 'our', 'website', 'send', 'your', 'password', 'send', 'us', 'your', 'account']
['send', 'us', 'your', 'password', 'review', 'our', 'website', 'account']


In [21]:
vocab_words_ham = []

for sentence in previous_ham:
    sentence_as_list = sentence.split()
    for word in sentence_as_list:
        vocab_words_ham.append(word)     

In [22]:
print(vocab_words_ham)
vocab_unique_words_ham = list(dict.fromkeys(vocab_words_ham))
print(vocab_unique_words_ham)

['Your', 'activity', 'report', 'benefits', 'physical', 'activity', 'the', 'importance', 'vows']
['Your', 'activity', 'report', 'benefits', 'physical', 'the', 'importance', 'vows']


In [23]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS #I have removed the stop words here

stop_words = set(map(str.lower, ENGLISH_STOP_WORDS))
filtered_words_SPAM = [word for word in vocab_unique_words_spam if word.lower() not in stop_words]
filtered_words_HAM = [word for word in vocab_unique_words_ham if word.lower() not in stop_words]

In [24]:
print(filtered_words_SPAM)
print(filtered_words_HAM)

['send', 'password', 'review', 'website', 'account']
['activity', 'report', 'benefits', 'physical', 'importance', 'vows']


In [31]:
dict_spamicity = {}
for w in filtered_words_SPAM:
    emails_with_w = 0     # counter
    for sentence in previous_spam:
        if w in sentence:
            print(w+":", sentence)
            emails_with_w+=1
            
    print(f"Number of spam emails with the word {w}: {emails_with_w}")
    total_spam = len(previous_spam)
    spamicity = (emails_with_w+1)/(total_spam+2)
    print(f"Spamicity of the word '{w}': {spamicity} \n")
    dict_spamicity[w.lower()] = spamicity

send: send us your password
send: send your password
send: send us your account
Number of spam emails with the word send: 3
Spamicity of the word 'send': 0.6666666666666666 

password: send us your password
password: send your password
Number of spam emails with the word password: 2
Spamicity of the word 'password': 0.5 

review: review our website
Number of spam emails with the word review: 1
Spamicity of the word 'review': 0.3333333333333333 

website: review our website
Number of spam emails with the word website: 1
Spamicity of the word 'website': 0.3333333333333333 

account: send us your account
Number of spam emails with the word account: 1
Spamicity of the word 'account': 0.3333333333333333 



In [39]:
dict_hamicity = {}
for w in filtered_words_HAM:
    emails_with_w = 0     # counter
    for sentence in previous_ham:
        if w in sentence:
            print(w+":", sentence)
            emails_with_w+=1
            
    print(f"Number of spam emails with the word {w}: {emails_with_w}")
    total_ham = len(previous_ham)
    hamicity = (emails_with_w+1)/(total_ham+2)
    print(f"hamicity of the word '{w}': {hamicity} \n")
    dict_hamicity[w.lower()] = hamicity       
    


activity: Your activity report
activity: benefits physical activity
Number of spam emails with the word activity: 2
hamicity of the word 'activity': 0.6 

report: Your activity report
Number of spam emails with the word report: 1
hamicity of the word 'report': 0.4 

benefits: benefits physical activity
Number of spam emails with the word benefits: 1
hamicity of the word 'benefits': 0.4 

physical: benefits physical activity
Number of spam emails with the word physical: 1
hamicity of the word 'physical': 0.4 

importance: the importance vows
Number of spam emails with the word importance: 1
hamicity of the word 'importance': 0.4 

vows: the importance vows
Number of spam emails with the word vows: 1
hamicity of the word 'vows': 0.4 



In [44]:
print("Dictionary Ham", dict_hamicity)
#
print("Dictionary Spam", dict_spamicity)

Dictionary Ham {'activity': 0.6, 'report': 0.4, 'benefits': 0.4, 'physical': 0.4, 'importance': 0.4, 'vows': 0.4}
Dictionary Spam {'send': 0.6666666666666666, 'password': 0.5, 'review': 0.3333333333333333, 'website': 0.3333333333333333, 'account': 0.3333333333333333}


In [35]:
prob_spam = len(previous_spam) / (len(previous_spam)+(len(previous_ham)))
print(prob_spam)

0.5714285714285714


<hr style=\"border:2px solid gray\"> </hr>

In [36]:
prob_ham = len(previous_ham) / (len(previous_spam)+(len(previous_ham)))
print(prob_ham)

0.42857142857142855


In [38]:
new_spam = [
    'send us your password',
    'transmit us your password',
    'transmit us your password',
    'submit us your password',
    'send us your password',
    'critique our website',
    'followup our website',
    'inspection our website',
    'review our website',
    'review our website',
    'send your password',
    'transmit your password',
    'transmit your password',
    'submit your password',
    'send your password',
    'send us your account',
    'transmit us your account',
    'transmit us your account',
    'submit us your account',
    'send us your account'
]

In [37]:
new_ham = [
    'Your activity report',
    'Your action report',
    'Your bodily function report',
    'Your activity report',
    'benefits physical activity',
    'welfare physical activity',
    'gains physical activity',
    'advantages physical activity',
    'benefits physical activity',
    'the importance vows',
    'the importance vows',
    'the importance vows',
    'the importance vows',
    'the importance vows',
    'Your activity report',
    'Your action report',
    'Your bodily function report',
    'Your activity report',
    'benefits physical activity',
    'gains physical activity'
]


In [51]:
tests = []
for i in new_emails['spam']:
    tests.append(i)
    
for i in new_emails['ham']:
    tests.append(i)
    
print(tests)    

['renew your password', 'renew your vows', 'benefits of our account', 'the importance of physical activity']


In [50]:
distinct_words_as_sentences_test = []

for sentence in tests:
    sentence_as_list = sentence.split()
    senten = []
    for word in sentence_as_list:
        senten.append(word)
    distinct_words_as_sentences_test.append(senten)
        
print(distinct_words_as_sentences_test)

[['renew', 'your', 'password'], ['renew', 'your', 'vows'], ['benefits', 'of', 'our', 'account'], ['the', 'importance', 'of', 'physical', 'activity']]


In [52]:
test_spam_tokenized = [distinct_words_as_sentences_test[0], distinct_words_as_sentences_test[1]]
test_ham_tokenized = [distinct_words_as_sentences_test[2], distinct_words_as_sentences_test[3]]
print(test_spam_tokenized)
print(test_ham_tokenized)

[['renew', 'your', 'password'], ['renew', 'your', 'vows']]
[['benefits', 'of', 'our', 'account'], ['the', 'importance', 'of', 'physical', 'activity']]


In [49]:
reduced_sentences_spam_test = []
for sentence in test_spam_tokenized:
    words_ = []
    for word in sentence:
        if word in vocab_unique_words_spam:
            print(f"'{word}', ok")
            words_.append(word)
        elif word in vocab_unique_words_ham:
            print(f"'{word}', ok")
            words_.append(word)
        else:
            print(f"'{word}', word not present in labelled spam training data")
    reduced_sentences_spam_test.append(words_)
print(reduced_sentences_spam_test)




'renew', word not present in labelled spam training data
'your', ok
'password', ok
'renew', word not present in labelled spam training data
'your', ok
'vows', ok
[['your', 'password'], ['your', 'vows']]


In [53]:
reduced_sentences_ham_test = []                   # repeat for ham words
for sentence in test_ham_tokenized:
    words_ = []
    for word in sentence:
        if word in vocab_unique_words_ham:
            print(f"'{word}', ok")
            words_.append(word)
        elif word in vocab_unique_words_spam:
            print(f"'{word}', ok")
            words_.append(word)
        else:
            print(f"'{word}', word not present in labelled ham training data")
    reduced_sentences_ham_test.append(words_)
print(reduced_sentences_ham_test)

'benefits', ok
'of', word not present in labelled ham training data
'our', ok
'account', ok
'the', ok
'importance', ok
'of', word not present in labelled ham training data
'physical', ok
'activity', ok
[['benefits', 'our', 'account'], ['the', 'importance', 'physical', 'activity']]


In [55]:
test_spam_stemmed = []
non_key = ['us',  'the', 'of','your']       # non-key words, gathered from spam,ham and test sentences
for email in reduced_sentences_spam_test:
    email_stemmed=[]
    for word in email:
        if word in non_key:
            print('remove')
        else:
            email_stemmed.append(word)
    test_spam_stemmed.append(email_stemmed)
            
print(test_spam_stemmed)

remove
remove
[['password'], ['vows']]


In [56]:
test_ham_stemmed = []
non_key = ['us',  'the', 'of', 'your'] 
for email in reduced_sentences_ham_test:
    email_stemmed=[]
    for word in email:
        if word in non_key:
            print('remove')
        else:
            email_stemmed.append(word)
    test_ham_stemmed.append(email_stemmed)
            
print(test_ham_stemmed)

remove
[['benefits', 'our', 'account'], ['importance', 'physical', 'activity']]


In [57]:
def mult(list_) :        # function to multiply all word probs together 
    total_prob = 1
    for i in list_: 
         total_prob = total_prob * i  
    return total_prob

In [59]:
def Bayes(email):
    probs = []
    for word in email:
        Pr_S = prob_spam
        print('prob of spam in general ',Pr_S)
        try:
            pr_WS = dict_spamicity[word]
            print(f'prob "{word}"  is a spam word : {pr_WS}')
        except KeyError:
            pr_WS = 1/(total_spam+2)  # Apply smoothing for word not seen in spam training data, but seen in ham training 
            print(f"prob '{word}' is a spam word: {pr_WS}")
            
        Pr_H = prob_ham
        print('prob of ham in general ', Pr_H)
        try:
            pr_WH = dict_hamicity[word]
            print(f'prob "{word}" is a ham word: ',pr_WH)
        except KeyError:
            pr_WH = (1/(total_ham+2))  # Apply smoothing for word not seen in ham training data, but seen in spam training
            print(f"WH for {word} is {pr_WH}")
            print(f"prob '{word}' is a ham word: {pr_WH}")
        
        prob_word_is_spam_BAYES = (pr_WS*Pr_S)/((pr_WS*Pr_S)+(pr_WH*Pr_H))
        print('')
        print(f"Using Bayes, prob the the word '{word}' is spam: {prob_word_is_spam_BAYES}")
        print('###########################')
        probs.append(prob_word_is_spam_BAYES)
    print(f"All word probabilities for this sentence: {probs}")
    final_classification = mult(probs)
    if final_classification >= 0.5:
        print(f'email is SPAM: with spammy confidence of {final_classification*100}%')
    else:
        print(f'email is HAM: with spammy confidence of {final_classification*100}%')
    return final_classification
for email in test_spam_stemmed:
    print('')
    print(f"           Testing stemmed SPAM email {email} :")
    print('                 Test word by word: ')
    all_word_probs = Bayes(email)
    print(all_word_probs)


           Testing stemmed SPAM email ['password'] :
                 Test word by word: 
prob of spam in general  0.5714285714285714
prob "password"  is a spam word : 0.5
prob of ham in general  0.42857142857142855
WH for password is 0.2
prob 'password' is a ham word: 0.2

Using Bayes, prob the the word 'password' is spam: 0.7692307692307692
###########################
All word probabilities for this sentence: [0.7692307692307692]
email is SPAM: with spammy confidence of 76.92307692307692%
0.7692307692307692

           Testing stemmed SPAM email ['vows'] :
                 Test word by word: 
prob of spam in general  0.5714285714285714
prob 'vows' is a spam word: 0.16666666666666666
prob of ham in general  0.42857142857142855
prob "vows" is a ham word:  0.4

Using Bayes, prob the the word 'vows' is spam: 0.35714285714285715
###########################
All word probabilities for this sentence: [0.35714285714285715]
email is HAM: with spammy confidence of 35.714285714285715%
0.3571428

## Reflection

Write you reflection in below cell.